# Introdução 

O objetivo deste notebook é a estatistica. Seja ela descritiva ou regressiva. Fazendo algumas definições, citando alguns teoremas importantes, ajustar dados, propor hipóteses e testa-las. 

A base de dados utilizada será a base PeNSE 2015 do [IBGE](https://www.ibge.gov.br/) sobre saude e educação, disponível em: 

> https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html?caminho=pense/2015/microdados/

A amostra que vamos utilizar é a `PeNSE_2015_AMOSTRA2.zip`

## PeNSE (Pesquisa Nacional de Saúde do Escolar)

A pesquisa PeNSE é uma pesquisa feita por amostragem que permite conhecer e dimensionar os fatores de risco e proteção à saúde dos adolescente , é feita utilizando dados para a seleção de escolas publicas e privadas do Instituto Nacional de Estudos e Pesquisas Educacionais Anísio Teixeira - INEP.

É fruto de uma parceria entre os Ministérios da Saúde e Educação. A pesquisa fornece informações sobre características básicas da população de estudo, incluindo aspectos socioecnômicos, como escolaridade dos pais, inserção no mercado de trabalho e posse de bens e serviços; fatores de risco comportamentais relacionados a hábitos alimentares, sedentarismo, tabagismo, consumo de álcool e outras drogas; saúde sexual e reprodutiva; exposição a acidentes e violências; hábitos de higiene; saúde bucal; saúde mental;... dentre vaŕias outras informações. 

### Tipo dos dados:
---

## Notas Metodológicas

---

## Importações de módulos

In [1]:
import pandas as pd 
import numpy as np 

## Importações de Dados

Aqui eu estou usando o Jupyter notebook mas em muitos casos pode-se utilizar o Google Colaboratory, onde é possível montar a imagem do Google Drive e fazer uso dos dados armazenados no Drive. Inclusive é possível armazenar um `.csv` dentro de um `.zip` e conseguir ler ele com o método `.read_csv()` do pandas. Para realizar o unZip do de um arquivo zip podemos usar a biblioteca `zipfile`.

> Lembrando que as ferramentas de Cloud podem ser pagas, ou seja, ter a capacidade de ler arquivos mesmo que zipados pode poupar dinheiro em algumas situações.

Outro método bastante importante do arquivo do tipo ZipFile é o `ZipFile.infolist()` que lista todos os arquivos dentro dele ou o `ZipFile.namelist()`

In [2]:
from zipfile import ZipFile

zip_file = ZipFile('./dados/PeNSE_2015_AMOSTRA2.zip')

for arquivo in zip_file.namelist():
    print(arquivo)

arquivos csv/
arquivos csv/PENSE_AMOSTRA2_ALUNO.CSV
arquivos csv/PENSE_AMOSTRA2_ALUNOESCOLA.CSV
arquivos csv/PENSE_AMOSTRA2_ESCOLA.CSV
arquivos sas/
arquivos sas/pense_amostra2_aluno.sas7bdat
arquivos sas/pense_amostra2_alunoescola.sas7bdat
arquivos sas/pense_amostra2_escola.sas7bdat
arquivos xlsx/
arquivos xlsx/PENSE_AMOSTRA2_ALUNO.xlsx
arquivos xlsx/PENSE_AMOSTRA2_ALUNOESCOLA.xlsx
arquivos xlsx/PENSE_AMOSTRA2_ESCOLA.xlsx
Dicionario_PENSE_Microdados_AMOSTRA2.xls


Note que podemos verificar exatamente quais arquivos compõem o zip. O `namelist()`é uma lista com o nome dos arquivos, então podemos selecionar um elemento desta lista e fornece-lo ao `zip_file.open(arquivo_desejado)`

In [3]:
caminho_arquivo_desejado = zip_file.namelist()[1]
caminho_arquivo_desejado

'arquivos csv/PENSE_AMOSTRA2_ALUNO.CSV'

In [4]:
dados = pd.read_csv(zip_file.open(caminho_arquivo_desejado), sep=';', thousands='.',decimal=',')

In [5]:
dados.head()

,ANOPESQ,PAIS,REGEOGR,VB00004,VB01001,VB01002,VB01003,VB01004,VB01005,VB01006,...,ESTRATO_EXP,ESTRATOGEOREG,PESO,V0006,V0007,V0008,V0041,aluno,escola,turma
0,2015,76,1,1,2,1,13,7,7,1,...,1223,1,299.735235,1,2,4,1,1,1,296
1,2015,76,1,1,2,1,14,4,7,1,...,1223,1,355.170081,1,2,4,1,2,1,296
2,2015,76,1,1,1,4,13,6,7,1,...,1223,1,299.735235,1,2,4,1,3,1,296
3,2015,76,1,1,1,1,14,6,7,1,...,1223,1,355.170081,1,2,4,1,4,1,296
4,2015,76,1,1,1,1,13,9,7,1,...,1223,1,299.735235,1,2,4,1,5,1,296


É possível também descompactar a pasta via terminal, seja ele numa célula do *jupyter* ou numa célula do *colaboratory*. 

In [6]:
dados.shape

(16556, 181)

Temos 16556 registros (linhas) e 181 variáveis (colunas).

Vamos analisar quais dados temos:

In [7]:
dados.columns

Index(['ANOPESQ', 'PAIS', 'REGEOGR', 'VB00004', 'VB01001', 'VB01002',
       'VB01003', 'VB01004', 'VB01005', 'VB01006',
       ...
       'ESTRATO_EXP', 'ESTRATOGEOREG', 'PESO', 'V0006', 'V0007', 'V0008',
       'V0041', 'aluno', 'escola', 'turma'],
      dtype='object', length=181)

Note que temos alguns códigos, para entender o que significa cada código precisamos de um dicionário de dados. Ou também podemos "adivinhar" os tipos de dados de cada variável

In [8]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16556 entries, 0 to 16555
Columns: 181 entries, ANOPESQ to turma
dtypes: float64(10), int64(171)
memory usage: 22.9 MB


Temos 7 variáveis do tipo `float64`, 171 variáveis do tipo `int64` e 3 variáveis do tipo `object`. Vamos investigar o que são essas variáveis do tipo object realizando um `select`na base de dados:

In [9]:
dados.select_dtypes(include='object')

""
0
1
2
3
4
...
16551
16552
16553
16554


> `VB17003`: Peso do estudante, incluindo os imputados, com uma casa decimal 

>`VB17004`:	Altura do estudante, incluindo os imputados, com uma casa decimal

Então vou alterar a importação dos dados adicionando os parâmetros: 

```python
dados = pd.read_csv(zip_file.open(caminho_arquivo_desejado), sep=';', thousands='.',decimal=',')
```

### Desafio: Classificar os dados em qualitativos e quantitativos. Discretos ou contínuos, nominais ou ordinais.

----

>`TEMPODESLOC`	Refere-se ao tempo médio diário acumulado pelo escolar, com o deslocamento da casa para escola e da escola para casa feito a pé ou de bicicleta, nos últimos sete dias anteriores à pesquisa. Em minutos.

>`TEMPOEDFIS`Refere-se ao tempo médio acumulado, nos últimos sete dias anteriores à pesquisa, que o escolar fez atividade física ou esporte durante as aulas de educação física na escola. Em minutos.

> `TEMPOEXTRA`	Refere-se ao tempo médio diário acumulado pelo escolar com a prática de alguma atividade física extraescolar como esportes, dança, ginástica, musculação, lutas ou outra atividade, nos últimos sete dias anteriores à data da pesquisa. Em minutos

> `TEMPOTOTAL`	A atividade física acumulada foi estimada calculando o produto entre o número de dias e o tempo médio que os escolares gastam em atividades físicas, nos sete dias anteriores à pesquisa, considerando os seguintes domínios: ir e voltar da escola, aulas de educação física e outras atividades extraescolares. Em minutos.

> `TEMPOEST`	A atividade física globalmente estimada refere-se ao número de dias que os escolares declararam fazer, pelo menos, uma hora por dia de atividade física, nos sete dias anteriores à pesquisa. Em minutos.

In [10]:
dicionario_dados = pd.read_excel('./dados/Dicionario_PENSE_Microdados_AMOSTRA2.xls')

In [11]:
dados['VB01001'].value_counts()

1    8287
2    8269
Name: VB01001, dtype: int64

### Variáveis que vamos seleciona:

> `VB01001`:	Qual é o seu sexo?
    > * 1	Masculino
    > * 2	Feminino

> `VB01002`:	Qual é a sua cor ou raça?
    > * 1	Branca
    > * 2	Preta
    > * 3	Amarela
    > * 4	Parda
    > * 5	Indígena
    > *99	Não informado
    
> `VB01022`	Em que turno você estuda?
    > * 1	Manhã
    > * 2	Intermediário
    > * 3	Tarde
    > * 4	Noite
    > * 5	Integral
    > * 99	Não informado

> `VB01013`	Na sua casa tem telefone fixo (convencional)?
    > * 1	Sim
    > * 2	Não
    > * 99	Não informado

> `VB01014`	Você tem celular?
    > * 1	SiM
    > * 2	Não
    > * 99	Não informado

> `VB01015A` Na sua casa tem computador (de mesa, ou netbook, laptop, etc)?
    > * 1	Sim
    > * 2	Não
    > * 99	Não informado

> `VB01016`	Você tem acesso à internet em sua casa?
    > * 1	Sim
    > * 2	Não
    > * 99	Não informado
    
> `VB01008A`	Qual nível de ensino (grau) sua mãe estudou ou estuda?
    > * 1	Minha mãe não estudou
    > * 2	Minha mãe começou o ensino fundamental ou 1º grau, mas não terminou
    > * 3	Minha mãe terminou o ensino fundamental ou 1º grau
    > * 4	Minha mãe começou o ensino médio ou 2º grau, mas não terminou
    > * 5	Minha mãe terminou o ensino médio ou 2° grau
    > * 6	Minha mãe começou a faculdade (ensino superior), mas não terminou
    > * 7	Minha mãe terminou a faculdade (ensino superior)
    > * 8	Não sei
    > * 99	Não informado
    
> `VB04006A`	Algum de seus pais ou responsáveis fuma? 
    > * -1	Pulo no questionário
    > * 1	Nenhum deles 
    > * 2	Só meu pai ou responsável do sexo masculino 
    > * 3	Só minha mãe ou responsável do sexo feminino 
    > * 4	Meu pai e minha mãe ou responsáveis 
    > * 5	Não sei
    > * 99	Não informado

> `VB05002`	Alguma vez na vida você tomou uma dose de bebida alcoólica? (Uma dose equivale a uma lata de cerveja ou uma taça de vinho ou uma dose de cachaça ou uísque etc)
    > * -1	Pulo no questionário
    > * 1	Sim
    > * 2	Não 
    > * 99	Não informado

> `VB07010`	Você já sofreu bullying?
    > * 1	Sim
    > * 2	Não
    > * 3	Não sei o que é bullying
    > * 99	Não informado

> `VB12001`	NOS ÚLTIMOS 12 MESES com que frequência tem se sentido sozinho(a)? 
    > * 1	Nunca
    > * 2	Raramente
    > * 3	Às vezes
    > * 4	Na maioria das vezes
    > * 5	Sempre
    > * 99	Não informado

> `VB12003`	Quantos amigos(as) próximos você tem?
    > * 1	Nenhum amigo (0)
    > * 2	1 amigo
    > * 3	2 amigos
    > * 4	3 ou mais amigos
    > * 99	Não informado

> `VB08001`	Você já teve relação sexual (transou) alguma vez?
    > * -1	Pulo no questionário
    > * 1	Sim
    > * 2	Não
    > * 99	Não informado

> `VB08011`	Você usou preservativo na primeira relação sexual?
    > * -1	Pulo no questionário
    > * 1	Sim
    > * 2	Não
    > * 99	Não informado

> `VB08008`	Na escola, você já recebeu orientação sobre prevenção de gravidez?
    > * -1	Pulo no questionário
    > * 1	Sim
    > * 2	Não
    > * 3	Não sei
    > * 99	Não informado

> `VB08009`	Na escola, você já recebeu orientação sobre AIDS ou outras Doenças Sexualmente Transmissíveis (DST)?
    > * -1	Pulo no questionário
    > * 1	Sim
    > * 2	Não
    > * 3	Não sei
    > * 99	Não informado

> `VB09016`	Alguma vez na vida você foi forçado a ter relação sexual? 
    > * -1	Pulo no questionário
    > * 1	Sim
    > * 2	Não
    > * 99	Não informado

> `VB13005`	Como você classificaria seu estado de saúde?
    > * 1	Muito bom
    > * 2	Bom
    > * 3	Regular
    > * 4	Ruim
    > * 5	Muito ruim
    > * 99	Não informado

> `VB11006`	Você considera sua imagem corporal como sendo algo:
    > * 1	Muito importante
    > * 2	Importante
    > * 3	Pouco importante
    > * 4	Sem importância
    > * 99	Não informado

> `VB11007`	Como você se sente em relação ao seu corpo?
    > * 1	Muito satisfeito(a)
    > * 2	Satisfeito(a)
    > * 3	Indiferente
    > * 4	Insatisfeito(a)
    > * 5	Muito insatisfeito(a)
    > * 99	Não informado

> VB17001	Qual é o peso do(a) aluno(a)?
    > * 0 a 2999	Peso
    > * 998	Não informado
    > * 9999	Recusa ou impossibilidade de medição

> `VB17002`	Qual é a altura do(a) aluno(a)?
    0 a 2999	Peso
    998	Não informado
    9999	Recusa ou impossibilidade de medição

> `VB17006`	Variável identificadora do estado nutricional do estudante, onde: déficit de peso=1
> * 1	Deficit de peso
> * 2	Eutrofia
> * 3	Sobrepeso
> * 4	Obesidade


> `V0006`	Situação da escola
    > * 1	Urbana
    > * 2	Rural

> `V0007`	Dependência Administrativa da escola
    > * 1	Pública
    > * 2	Privada

> `V0008`	Esfera Administrativa da escola
    > * 1	Municipal
    > * 2	Estadual
    > * 3	Federal
    > * 4	Privada
    
> `V0041`	Tipo de escola privada
    > * -1	Pulo no questionário
    > * 1	Particular
    > * 2	Comunitária
    > * 3	Confessional
    > * 4	Filantrópica

In [12]:
selecao = {'VB01001':'sexo',
           'VB01002':'cor',
           'VB01022':'turno',
           'VB01013':'telefone fixo',
           'VB01014':'celular',
           'VB01015A':'computador',
           'VB01016':'internet',
           'VB01008A':'escolaridade mae',
           'VB04006A':'responsáveis fuma',
           'VB05002':'usou bebida alcoolica',
           'VB07010':'sofreu bullying',
           'VB12001':'frequência com que se sente só',
           'VB12003':'Quantos amigos próximos',
           'VB08001':'teve relação sexual alguma vez',
           'VB08011':'usou preservativo na primeira relação',
           'VB08009':'ja recebeu orientação sobre DST na escola',
           'VB09016':'ja foi forçado a ter relação sexual', 
           'VB13005':'como classifica o proprio estado de saúde',
           'VB11006':'considera sua imagem corporal como sendo algo',
           'VB11007':'como se sente em relação ao seu corpo',
           'VB17001':'peso do aluno',
           'VB17002':'altura do aluno',
           'VB17006':'estado nutricional do estudante',
           'V0006':'Situação da escola',
           'V0007':'Dependência Administrativa da escola',
           'V0008':'Esfera Administrativa da escola',
           'V0041':'Tipo de escola privada'
          }

In [13]:
dados_selecionados = dados[selecao.keys()]
dados_selecionados.columns = selecao.values()

In [14]:
dados_selecionados

,sexo,cor,turno,telefone fixo,celular,computador,internet,escolaridade mae,responsáveis fuma,usou bebida alcoolica,...,como classifica o proprio estado de saúde,considera sua imagem corporal como sendo algo,como se sente em relação ao seu corpo,peso do aluno,altura do aluno,estado nutricional do estudante,Situação da escola,Dependência Administrativa da escola,Esfera Administrativa da escola,Tipo de escola privada
0,2,1,1,1,1,1,1,7,1,2,...,1,1,2,371.0,1570.0,1,1,2,4,1
1,2,1,1,1,1,1,1,7,2,2,...,3,3,2,502.0,1520.0,2,1,2,4,1
2,1,4,1,2,1,1,1,5,1,1,...,2,1,2,482.0,1614.0,2,1,2,4,1
3,1,1,1,1,1,1,1,7,1,2,...,1,1,2,694.0,1725.0,3,1,2,4,1
4,1,1,1,2,1,1,1,5,1,1,...,2,1,4,789.0,1675.0,4,1,2,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16551,2,4,1,1,1,1,1,7,-1,-1,...,1,1,1,567.0,1679.0,2,1,2,4,1
16552,1,1,1,2,1,1,1,7,1,2,...,2,1,2,619.0,1858.0,2,1,2,4,1
16553,1,4,1,1,2,1,1,7,2,2,...,1,4,1,609.0,1594.0,3,1,2,4,1
16554,1,4,1,1,1,1,1,7,2,1,...,1,3,2,473.0,1578.0,2,1,2,4,1


In [15]:
dados_selecionados['sexo'].value_counts(normalize=True) * 100 

1    50.054361
2    49.945639
Name: sexo, dtype: float64

Note que a proporção entre homens e mulheres esta normal.

### Vamos montar tabelas de frequencias e percentuais

> `VB11007`	Como você se sente em relação ao seu corpo?
    > * 1	Muito satisfeito(a)
    > * 2	Satisfeito(a)
    > * 3	Indiferente
    > * 4	Insatisfeito(a)
    > * 5	Muito insatisfeito(a)
    > * 99	Não informado

In [16]:
percentual_satisfacao_corpo = dados_selecionados['como se sente em relação ao seu corpo'].value_counts(normalize=True)
frequencia_satisfacao_corpo = dados_selecionados['como se sente em relação ao seu corpo'].value_counts()

distribucao_frequencia_satisfacao_corpo = pd.DataFrame({'frequencia': frequencia_satisfacao_corpo,
                                                       'percentual': percentual_satisfacao_corpo * 100})

respostas_satisfacao_corpo = {
    1:'Muito satisfeito(a)',
    2:'Satisfeito(a)',
    3:'Indiferente',
    4:'Insatisfeito(a)',
    5:'Muito insatisfeito(a)',
    99:'Não informado'
}

distribucao_frequencia_satisfacao_corpo.rename(index=respostas_satisfacao_corpo, inplace=True)
distribucao_frequencia_satisfacao_corpo.rename_axis(index='Sentimento em relação ao corpo')

,frequencia,percentual
Sentimento em relação ao corpo,,
Satisfeito(a),6920,41.797536
Muito satisfeito(a),4608,27.832810
Insatisfeito(a),2326,14.049287
Indiferente,1807,10.914472
Muito insatisfeito(a),714,4.312636
Não informado,181,1.093259


### Desafio: 
> Criar uma função para gerar as tabelas e fazer uma análise das variáveis escolhidas.

In [17]:
def cria_tabela_frequencia_percentuais(series, classificacoes,rotulo):
    percentual = series.value_counts(normalize=True) * 100
    frequencia = series.value_counts()
    
    tab1 = pd.DataFrame({'frequencia':frequencia,
                        'percentual':percentual})
    tab1.rename(index=classificacoes, inplace=True)
    tab1.rename_axis(index=rotulo, inplace=True)
    return tab1
    

In [18]:
### Testando
turno = {1: 'Manhã',2: 'Intermediário',3: 'Tarde',4: 'Noite',5: 'Integral',99: 'Não informado'}

cria_tabela_frequencia_percentuais(dados_selecionados['turno'], turno, 'Turno em que estuda')

,frequencia,percentual
Turno em que estuda,,
Manhã,9198,55.556898
Tarde,5362,32.387050
Noite,1258,7.598454
Integral,651,3.932109
Intermediário,83,0.501329
Não informado,4,0.024160


#### Construindo classificadores

In [19]:
sexo = {1: 'Masculino',2: 'Feminino'}
cor = {1:'Branca',2:'Preta',3:'Amarela',4:'Parda',5:'Indígena',99:'Não informado'}
turno = {1: 'Manhã',2: 'Intermediário',3: 'Tarde',4: 'Noite',5: 'Integral',99: 'Não informado'}

telefone_fixo = {1:'Sim',2:'Não',99:'Não informado'}
celular = {1:'Sim',2:'Não',99:'Não informado'}
computador = {1:'Sim',2:'Não',99:'Não informado'}
internet = {1:'Sim',2:'Não',99:'Não informado'}

escolaridade_mae = {
    1: 'Minha mãe não estudou',
    2: 'Minha mãe começou o ensino fundamental ou 1º grau, mas não terminou',
    3: 'Minha mãe terminou o ensino fundamental ou 1º grau',
    4: 'Minha mãe começou o ensino médio ou 2º grau, mas não terminou',
    5: 'Minha mãe terminou o ensino médio ou 2° grau',
    6: 'Minha mãe começou a faculdade (ensino superior), mas não terminou',
    7: 'Minha mãe terminou a faculdade (ensino superior)',
    8: 'Não sei',
    99: 'Não informado'
}

responsaveis_fuma = {
    -1: 'Pulo no questionário',
    1: 'Nenhum deles',
    2: 'Só meu pai ou responsável do sexo masculino',
    3: 'Só minha mãe ou responsável do sexo feminino',
    4: 'Meu pai e minha mãe ou responsáveis',
    5:'Não sei',
    99:'Não informado'
}
usou_bebida_alcoolica = {
    -1: 'Pulo no questionário',
    1: 'Sim',
    2: 'Não',
    99:'Não informado'
}
sofreu_bullying = {
    1: 'Sim',
    2: 'Não',
    3: 'Não sei o que é bullying',
    99:'Não informado'
}
frequência_com_que_se_sente_isolado = {
    1:'Nunca',
    2:'Raramente',
    3: 'Às vezes',
    4: 'Na maioria das vezes',
    5:'Sempre',
    99:'Não informado'
}
Quantos_amigos_proximos = {
    1:'Nenhum amigo (0)',
    2: '1 amigo',
    3: '2 amigos',
    4: '3 ou mais amigos',
    99:'Não informado'
}
teve_relação_sexual_alguma_vez = {
    -1: 'Pulo no questionário',
    1: 'Sim',
    2: 'Não',
    99:'Não informado'
}
usou_preservativo_na_primeira_relacao = {
    -1: 'Pulo no questionário',
    1: 'Sim',
    2: 'Não',
    99:'Não informado'
}
ja_recebeu_orientação_sobre_DST_na_escola = {
    -1: 'Pulo no questionário',
    1: 'Sim',
    2: 'Não',
    99:'Não informado'
}
ja_foi_forçado_a_ter_relação_sexual = {
    -1: 'Pulo no questionário',
    1: 'Sim',
    2: 'Não',
    99:'Não informado'
}
como_classifica_o_proprio_estado_de_saude = {
    1:'Muito bom',
    2: 'Bom',
    3: 'Regular',
    4: 'Ruim',
    5: 'Muito ruim',
    99: 'Não informado'
}
considera_sua_imagem_corporal_como_sendo_algo = {
    1:'Muito importante',
    2:'Importante',
    3:'Pouco importante',
    4:'Sem importância',
    99: 'Não informado'
}
como_se_sente_em_relação_ao_seu_corpo = {
    1: 'Muito satisfeito(a)',
    2: 'Satisfeito(a)',
    3: 'Indiferente',
    4:'Insatisfeito(a)',
    5:'Muito insatisfeito(a)',
    99:'Não informado'
}
peso_do_aluno = {}
altura_do_aluno = {}
estado_nutricional_do_estudante = {
    1: 'Deficit de peso',
    2: 'Eutrofia',
    3: 'Sobrepeso',
    4: 'Obesidade'
}
Situacao_da_escola = {1:'Urbana',2: 'Rural'}
Dependencia_Administrativa_da_escola = {1:'Pública',2:'Privada'}
Esfera_Administrativa_da_escola = {1: 'Municipal',2: 'Estadual',3: 'Federal', 4: 'Privada'}
Tipo_de_escola_privada = {
    -1: 'Pulo no questionário',
    1: 'Particular',
    2: 'Comunitária',
    3: 'Confessional',
    4:'Filantrópica'}



In [20]:
tabelas_de_variaveis = [sexo, cor,turno,telefone_fixo,celular,computador,
                 internet,escolaridade_mae, responsaveis_fuma, usou_bebida_alcoolica,
                 sofreu_bullying, frequência_com_que_se_sente_isolado, Quantos_amigos_proximos,
                 teve_relação_sexual_alguma_vez, usou_preservativo_na_primeira_relacao,
                 ja_recebeu_orientação_sobre_DST_na_escola, ja_foi_forçado_a_ter_relação_sexual,
                 como_classifica_o_proprio_estado_de_saude, considera_sua_imagem_corporal_como_sendo_algo,
                 peso_do_aluno, altura_do_aluno, estado_nutricional_do_estudante,
                 Situacao_da_escola, Dependencia_Administrativa_da_escola, 
                 Esfera_Administrativa_da_escola, Tipo_de_escola_privada]

dicionario_variaveis = {}
i = 0
for variavel in dados_selecionados.columns:
    dicionario_variaveis[str(variavel)] = tabelas_de_variaveis[i]
    i += 1

IndexError: list index out of range

In [21]:
dicionario_variaveis

{'sexo': {1: 'Masculino', 2: 'Feminino'},
 'cor': {1: 'Branca',
  2: 'Preta',
  3: 'Amarela',
  4: 'Parda',
  5: 'Indígena',
  99: 'Não informado'},
 'turno': {1: 'Manhã',
  2: 'Intermediário',
  3: 'Tarde',
  4: 'Noite',
  5: 'Integral',
  99: 'Não informado'},
 'telefone fixo': {1: 'Sim', 2: 'Não', 99: 'Não informado'},
 'celular': {1: 'Sim', 2: 'Não', 99: 'Não informado'},
 'computador': {1: 'Sim', 2: 'Não', 99: 'Não informado'},
 'internet': {1: 'Sim', 2: 'Não', 99: 'Não informado'},
 'escolaridade mae': {1: 'Minha mãe não estudou',
  2: 'Minha mãe começou o ensino fundamental ou 1º grau, mas não terminou',
  3: 'Minha mãe terminou o ensino fundamental ou 1º grau',
  4: 'Minha mãe começou o ensino médio ou 2º grau, mas não terminou',
  5: 'Minha mãe terminou o ensino médio ou 2° grau',
  6: 'Minha mãe começou a faculdade (ensino superior), mas não terminou',
  7: 'Minha mãe terminou a faculdade (ensino superior)',
  8: 'Não sei',
  99: 'Não informado'},
 'responsáveis fuma': {-1: '

In [23]:
### Testando
dataframes_com_percentuais_frequencias_variaveis = {}
for rotulo, variavel in dicionario_variaveis.items():
    dataframe = cria_tabela_frequencia_percentuais(dados_selecionados[rotulo], variavel, str(rotulo))
    display(dataframe)
    dataframes_com_percentuais_frequencias_variaveis[rotulo] = dataframe

,frequencia,percentual
sexo,,
Masculino,8287,50.054361
Feminino,8269,49.945639


,frequencia,percentual
cor,,
Parda,6726,40.625755
Branca,6575,39.713699
Preta,1939,11.711766
Amarela,712,4.300556
Indígena,581,3.509302
Não informado,23,0.138922


,frequencia,percentual
turno,,
Manhã,9198,55.556898
Tarde,5362,32.387050
Noite,1258,7.598454
Integral,651,3.932109
Intermediário,83,0.501329
Não informado,4,0.024160


,frequencia,percentual
telefone fixo,,
Não,8479,51.214061
Sim,8057,48.665137
Não informado,20,0.120802


,frequencia,percentual
celular,,
Sim,14366,86.772167
Não,2172,13.119111
Não informado,18,0.108722


...

,frequencia,percentual
altura do aluno,,
998.0,385,2.332061
9999.0,265,1.605185
1600.0,186,1.126658
1620.0,153,0.926767
1570.0,146,0.884366
...,...,...
703.0,1,0.006057
1378.0,1,0.006057
1862.0,1,0.006057


,frequencia,percentual
estado nutricional do estudante,,
Rural,11479,69.334380
3,3028,18.289442
4,1514,9.144721
Urbana,535,3.231457


,frequencia,percentual
Situação da escola,,
Pública,15705,94.85987
Privada,851,5.14013


,frequencia,percentual
Dependência Administrativa da escola,,
Municipal,12381,74.782556
Estadual,4175,25.217444


,frequencia,percentual
Esfera Administrativa da escola,,
Comunitária,9009,54.415318
Filantrópica,4175,25.217444
Confessional,3072,18.555207
Particular,300,1.812032
